In [1]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

Mounted at /content/drive
Mounted at /content/gdrive


In [7]:
from numpy import save, load
from pandas import read_csv
from tensorflow.keras import layers
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras import Model
from keras.layers import Conv1D, SpatialDropout1D
from keras.layers import Convolution1D, Dense,Activation
from keras.models import Input, Model
from keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,GlobalAveragePooling1D,TimeDistributed, MaxPooling1D
import tensorflow as tf
import numpy as np
import keras.backend as K
import keras.layers
from keras import optimizers
from keras.layers import Activation, Lambda
from keras.layers import Convolution1D, Dense
from keras.models import Input, Model
from typing import List, Tuple
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
from keras.wrappers.scikit_learn import KerasClassifier

In [3]:
import numpy as np
x_train = np.load('gdrive/My Drive/dataset/dataset2/trainX.npy')
y_train = np.load('gdrive/My Drive/dataset/dataset2/trainy.npy')
x_test = np.load('gdrive/My Drive/dataset/dataset2/testX.npy')
y_test = np.load('gdrive/My Drive/dataset/dataset2/testy.npy')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(44339, 128, 6) (44339, 20) (4936, 128, 6) (4936, 20)


In [4]:
x_train = x_train.transpose(0,2,1)
x_test = x_test.transpose(0,2,1)

In [5]:
# Merge inputs and targets
inputs = np.concatenate((x_train, x_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)

In [8]:
embed_dim = x_train.shape[-1]   # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = x_train.shape[-1]  # Hidden layer size in feed forward network inside transformer

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads,
                                             key_dim=embed_dim)
        self.ffn = keras.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)  # self-attention layer
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)  # layer norm
        ffn_output = self.ffn(out1)  #feed-forward layer
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)  # layer norm

In [9]:
# Define per-fold score containers 
acc_per_fold = []
loss_per_fold = []
num_folds = 5
kfold = KFold(n_splits=num_folds, shuffle=True)
fold_no = 1
input_shape = x_train.shape[1:]
input = Input(shape=x_train.shape[1:])
for train, test in kfold.split(inputs, targets):

  model = keras.Sequential()
  model.add(layers.Input(shape= x_train.shape[1:]))
  model.add(TransformerBlock(embed_dim, num_heads, ff_dim))
  model.add(layers.GlobalAveragePooling1D())
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(ff_dim, activation='relu'))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(20, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')
  history = model.fit(inputs[train], targets[train], epochs=200, verbose=True, batch_size=128,callbacks=[keras.callbacks.EarlyStopping(monitor='loss', patience=5,restore_best_weights=True)])
    # Generate generalization metrics
  scores = model.evaluate(inputs[test], targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

model.summary()



------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/200
308/308 [==============================] - 8s 13ms/step - loss: 0.9026 - accuracy: 0.7452
Epoch 2/200
308/308 [==============================] - 4s 12ms/step - loss: 0.3155 - accuracy: 0.9128
Epoch 3/200
308/308 [==============================] - 4s 14ms/step - loss: 0.2437 - accuracy: 0.9311
Epoch 4/200
308/308 [==============================] - 4s 13ms/step - loss: 0.1931 - accuracy: 0.9458
Epoch 5/200
308/308 [==============================] - 4s 13ms/step - loss: 0.1723 - accuracy: 0.9508
Epoch 6/200
308/308 [==============================] - 4s 13ms/step - loss: 0.1565 - accuracy: 0.9545
Epoch 7/200
308/308 [==============================] - 4s 12ms/step - loss: 0.1401 - accuracy: 0.9584
Epoch 8/200
308/308 [==============================] - 4s 13ms/step - loss: 0.1270 - accuracy: 0.9625
Epoch 9/200
308/308 [==============================] - 4s 12ms/step - loss: 0.1163 - ac

In [10]:
# Average scores 
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.17365725338459015 - Accuracy: 96.19482755661011%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.15621525049209595 - Accuracy: 96.77321314811707%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.11356059461832047 - Accuracy: 97.11821675300598%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.14728029072284698 - Accuracy: 96.47894501686096%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.11257883906364441 - Accuracy: 97.22983241081238%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 96.7590069770813 (+- 0.3866893294387845)
> Loss: 0.1406584456562996
------------------------